In [9]:
import sys
sys.path.append('..')
from scripts.predict import get_location_data
from scripts.download_data import get_entries_since, preprocessing
from google.cloud import bigquery
import numpy as np
import pickle

In [3]:
locations_df = get_location_data('../data/raw/channels.csv', ['location', 'id', 'lat', 'long'])
locations_df.head()

,location,id,lat,long
0,Ntinda,930434,0.360209,32.610756
1,Bugolobi,718028,0.307500,32.620600
2,Nantabulirwa,912224,0.346460,32.703280
3,Kireka,930426,0.365500,32.646800
4,Luwafu,930427,0.268900,32.588000


In [4]:
channels = list(locations_df.id)
len(channels)

37

In [5]:
#Arrays to hold X and Y data
X = np.zeros([0,3])
Y = np.zeros([0,1])

In [6]:
for chan in channels:
        d = get_entries_since(chan)
        if d.shape[0]!=0:
            d = preprocessing(d)
            #loc = get_loc(chan)
            loc = locations_df.loc[locations_df['id'] == chan, ['id', 'lat', 'long']]
            loc = loc[['id', 'long', 'lat']]
                
            Xchan = np.c_[np.repeat(np.array(loc)[:,1:],d.shape[0],0),[n.timestamp()/3600 for n in d['created_at']]]
                
            Ychan = np.array(d['pm2_5'])
            X = np.r_[X,Xchan]#appending device X data to array X
            Y = np.r_[Y,Ychan[:, None]]#appending device Y data to array Y
            print (str(chan)+':done!')
        else:
            print(str(chan)+':empty!')

930434:done!
718028:done!
912224:done!
930426:done!
930427:done!
912223:done!
912222:empty!
912220:done!
870145:done!
870143:done!
870144:empty!
870147:done!
870142:done!
870139:done!
832255:empty!
832252:empty!
832253:done!
832254:done!
832251:empty!
782720:done!
782719:done!
782718:done!
755614:done!
755612:done!
870146:done!
737276:empty!
737273:done!
689761:done!
718029:done!
718030:done!
730014:done!
782721:done!
782722:done!
912219:empty!
912221:done!
912225:done!
930429:done!


In [11]:
pickle.dump({'X':X,'Y':Y},open('../data/raw/data.p','wb'))